In [ ]:
"""
Author: maxwell.cook@colorado.edu
"""

import os, sys

# Custom functions
sys.path.append(os.path.join(os.getcwd(),'code/'))
from __functions import *

albers = 'EPSG:5070' # albers CONUS
utm = 'EPSG:32613' # UTM Zone 13N

maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/'
projdir = os.path.join(maindir, 'aspen-fire/Aim2/')

print("Ready to go !")

In [ ]:
# load the aggregated FRP grids (regular 375m2 grids summarizing FRP from VIIRS)
fp = os.path.join(projdir,'data/spatial/mod/VIIRS/viirs_snpp_jpss1_afd_latlon_fires_pixar_gridstats.gpkg')
grid = gpd.read_file(fp)
print(f"\nThere are [{len(grid)}] grids across [{len(grid['Fire_ID'].unique())}] fires.\n")

# create a unique ID
grid['grid_idx'] = grid['Fire_ID'].astype(str) + grid['grid_index'].astype(str)

# add the centroid lat/lon to the grid data
df = grid.to_crs(4326) # WGS coords for lat/lon
df['x'] = df.geometry.centroid.x  # Longitude (x-coordinate)
df['y'] = df.geometry.centroid.y  # Latitude
grid = grid.merge(df[['grid_idx','x','y']], on='grid_idx', how='left')
del df
print(f"\n{grid.columns}\n")

# Drop any duplicate grids ...
print(f"Dropping [{grid.duplicated(subset=['grid_idx','Fire_ID']).sum()}] duplicate grids.\n")
grid = grid.drop_duplicates(subset=['grid_idx','Fire_ID'], keep='first')

In [ ]:
# Check out the distribution of grid overlap with FRP observations
thresh = 0.50
print(f"Fractional overlap:\n{grid['overlap'].describe()}\n")
n_small = grid[grid['overlap'] < thresh]['grid_idx'].count() 

# Plot the distribution of the fractional overlap
plt.figure(figsize=(6,3))
sns.histplot(grid['overlap'], kde=True, bins=50, color='dodgerblue', alpha=0.7)

# Add vertical line for the threshold and for 100%
plt.axvline(x=thresh, color='red', linestyle='--', label=f'{thresh*100}% Threshold')
plt.axvline(x=1, color='grey', linestyle='--', label='100% Overlap')

# Customize the plot
plt.xlabel('Cumulative Fractional Overlap')
plt.ylabel('Frequency')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.5)
plt.text(16.5, plt.ylim()[1] * 0.7, 
         f'N = {n_small} [{round(n_small/len(grid)*100,2)}%]', 
         fontsize=10, color='black')

# Save the plot
out_path = os.path.join(projdir, 'figures/grid_overlap_distribution.png')
plt.savefig(out_path, dpi=300, bbox_inches='tight')

plt.show()
print(f"Plot saved to: {out_path}")

# filter out grids below the overlap threshold
print(f"\nDropping [{n_small} ({round(n_small/len(grid)*100,2)}%)] grids with <{thresh*100}% fractional overlap.")
grid = grid[grid['overlap'] >= thresh] # remove these observations
print(len(grid))

## Gather LANDFIRE variables for the study region

LANDFIRE ca. 2016; [EVT, CC, CH, CBD, CBH, ]

In [ ]:
fuel_vars = {
    'CC': 'https://lfps.usgs.gov/arcgis/rest/services/Landfire_LF200/US_200CC_19/ImageServer',
    'CH': 'https://lfps.usgs.gov/arcgis/rest/services/Landfire_LF200/US_200CH_19/ImageServer',
    'CBD': 'https://lfps.usgs.gov/arcgis/rest/services/Landfire_LF200/US_200CBD_19/ImageServer',
    'CBH': 'https://lfps.usgs.gov/arcgis/rest/services/Landfire_LF200/US_200CBH_19/ImageServer'
}

# reproject the grid and get the bounds
grid_wgs = grid.to_crs('EPSG:4326')

# grab the EVT raster to use for zones
evt_url = 'https://lfps.usgs.gov/arcgis/rest/services/Landfire_LF200/US_200EVT/ImageServer'
print(f"Downloading the EVT raster from: {evt_url}")
evt_da = get_image_service_array(evt_url, grid_wgs, out_prefix='EVT')

# crop to gridcells
# match the CRS
grid_prj = grid.to_crs(evt_da.rio.crs)
# create the crop extent
bbox = box(*grid_prj.total_bounds) # make a bounding box
bbox = gpd.GeoDataFrame(geometry=[bbox], crs='EPSG:5070')
evt_da = evt_da.rio.clip(bbox.geometry) # do the crop

# get a list of EVT values
evt_arr = evt_da.values
evt_nodata = evt_da.rio.nodata
evt_vals = np.unique(evt_arr)
evt_vals = evt_vals[~np.isnan(evt_vals) & (evt_vals != evt_nodata)]
print(f"Found [{len(evt_vals)}] EVT codes")
del evt_nodata, evt_arr

In [ ]:
# identify EVT codes of interest
# first, download the data dictionary
dd = list_files(os.getcwd(),'*EVT.csv',recursive=True)[0]
print(dd)
if not dd:
    print("Downloading data dictionary")
    dd = download_lf_csv('EVT')
else:
    dd = pd.read_csv(dd)
# merge with the codes
evt_vals_df = pd.DataFrame({'EVT': evt_vals.astype(int)})
evt_df = evt_vals_df.merge(dd, left_on='EVT', right_on='VALUE')
evt_df

In [ ]:
evt_df.columns

In [ ]:
evt_df['EVT_SBCLS'].unique()

In [ ]:
# filter to forested EVTs, look at the SAF code
trees_shrubs = ['Deciduous open tree canopy', 'Evergreen open tree canopy',
                'Evergreen closed tree canopy', 'Mixed evergreen-deciduous open tree canopy',
                'Evergreen sparse tree canopy']
evt_df_tree = evt_df[evt_df['EVT_SBCLS'].isin(trees_shrubs)]
evt_df_tree['SAF_SRM'].unique()

In [ ]:
# filter to species of interest using the SAF code
saf_keep = [
    'SAF 217: Aspen',
    'SRM 504: Juniper-Pinyon Pine Woodland',
    'SAF 218: Lodgepole Pine',
    'SAF 210: Interior Douglas-Fir',
    'SAF 211: White Fir',
    'SAF 237: Interior Ponderosa Pine',
    'SAF 206: Engelmann Spruce-Subalpine Fir',
    'LF 41: Deciduous Shrubland'
]

# gather new set of EVT codes:
evt_df_tree_evt = evt_df_tree[evt_df_tree['SAF_SRM'].isin(saf_keep)]
print(f"Filtered to [{len(evt_df_tree_evt['EVT'].unique())}] EVT codes")

In [ ]:
# get the fuels arrays
print("\nAccessing fuel rasters")

fuel_arrs = {} # to store the results
for var, url in fuel_vars.items():
    print(f"\nDownloading [{var}] raster from: {url}")
    # get the raster
    da = get_image_service_array(url, grid_wgs, out_prefix=var)
    # crop it using the bounding box we created earlier
    da = da.rio.clip(bbox.geometry) # crop to the extent bounds
    # add to the results dictionary
    fuel_arrs[var] = da
    del da

print("\nFuel rasters successfully processed")

### Gather the USFS TreeMap bands (ca. 2016)

BALIVE, SDI, and QMD

In [ ]:
# load the TreeMap rasters (locally stored)
tm_paths = {
    'BALIVE': list_files(os.path.join(maindir, 'data/landcover/USFS/RDS_TreeMap/'), ext='*_BALIVE.tif', recursive=True)[0],
    'SDI': list_files(os.path.join(maindir, 'data/landcover/USFS/RDS_TreeMap/'), ext='*SDI*.tif', recursive=True)[0],
    'QMD': list_files(os.path.join(maindir, 'data/landcover/USFS/RDS_TreeMap/'), ext='*QMD*.tif', recursive=True)[0]
}
print(tm_paths, "\n")

# store the cropped arrays
tm_arrs = {}
ref = next(iter(fuel_arrs.values()))  # reference raster
for tm, fp in tm_paths.items():
    print(f"\nProcessing [{tm}]")
    # open the raster files
    with rio.open(fp) as src:
        bounds = grid.to_crs(src.crs).total_bounds
        window = from_bounds(*bounds, transform=src.transform)
        data = src.read(1, window=window)

        # Build spatial coordinates
        transform = src.window_transform(window)
        height, width = data.shape
        x_coords = np.arange(width) * transform.a + transform.c + transform.a / 2
        y_coords = np.arange(height) * transform.e + transform.f + transform.e / 2

        da = xr.DataArray(
            data,
            dims=("y", "x"),
            coords={"y": y_coords, "x": x_coords},
            name=tm
        )
        da.rio.write_crs(src.crs, inplace=True)
        da.rio.write_transform(transform, inplace=True)

        # reproject to match the reference
        da_prj = da.rio.reproject_match(ref)

        tm_arrs[tm] = da_prj # add to the array dictionary
        del da, da_prj

print("\nProcessing TreeMap layers complete.")

### Create the data stack and convert to dataframe summary

Stack the EVT and fuels arrays, the TreeMap bands, and a rasterized 'grid_idx' into one multiband array. Then, convert to a data frame and calculate the EVT-specific fuel conditions for each 'grid_idx'.

In [ ]:
# create a raster grid for grid_idx to use as zones
# convert grid_idx to numeric
grid_prj['grid_idx'] = pd.to_numeric(grid_prj['grid_idx'], errors='coerce')
grid_idx_da = rasterize_grid_idx(grid_prj, ref_da=evt_da)
print("Created the 'grid_idx' raster")
print(f"Output shape: {grid_idx_da.shape}")
print(f"Reference image shape: {evt_da.shape}")

In [ ]:
# stack the bands
da_stack = xr.Dataset({
    var: da
    for var, da in fuel_arrs.items()
})

# add the EVT, TreeMap and grid_idx
da_stack['EVT'] = evt_da
da_stack['BALIVE'] = tm_arrs['BALIVE']
da_stack['SDI'] = tm_arrs['SDI']
da_stack['QMD'] = tm_arrs['QMD']
da_stack['grid_idx'] = grid_idx_da

da_stack

In [ ]:
# gather the dataframe
df = da_stack.to_dataframe().reset_index()
df = df.dropna(subset=['EVT']) # keep valid rows
df = df[df['grid_idx'] != 0] # filter where grid_idx != 0 (background)
# filter to EVT codes of interest
df = df[df['EVT'].isin(evt_df_tree_evt['EVT'].unique())]
df['EVT'] = df['EVT'].astype(int) # force EVT code to integer

# group and summarize by grid_idx
df = df.groupby(['grid_idx','EVT']).agg({
    'CC': 'mean',
    'CH': 'mean',
    'CBH': 'mean',
    'CBD': 'mean',
    'BALIVE': 'mean',
    'SDI': 'mean',
    'QMD': 'mean'
}).reset_index()
df.head()

In [ ]:
# join the EVT names
dd_ = dd[['VALUE','SAF_SRM']]
grid_evt = df.merge(dd_, left_on='EVT', right_on='VALUE')
grid_evt

In [ ]:
print(len(grid_evt))
print(grid_evt['SDI'].isna().sum())
grid_evt[grid_evt['SDI'].isna()]['SAF_SRM'].value_counts()

In [ ]:
# manage the EVT values, reclassing
grid_evt['SAF_SRM'].unique()

In [ ]:
# Aggregate species into forest groups
# These groups represent common pairings for the Southern Rockies
spp_grouping = {
    'SAF 217: Aspen': 'Aspen',
    'SAF 206: Engelmann Spruce-Subalpine Fir': 'Spruce-fir',
    'SAF 237: Interior Ponderosa Pine': 'Ponderosa pine',
    'SAF 218: Lodgepole Pine': 'Lodgepole pine',
    'SAF 210: Interior Douglas-Fir': 'Douglas-fir',
    'SAF 211: White Fir': 'White fir',
    'SRM 504: Juniper-Pinyon Pine Woodland': 'Pinon-juniper',
}

# create the remap table
spp_remap = {} # dictionary to store the remap values
# Iterate over groups to create the species remap dictionary
for keywords, spp_group in spp_grouping.items():
    # Find species matching the keywords
    spp = grid_evt[grid_evt['SAF_SRM'].str.contains(keywords, case=False, na=False)]
    # Add matching species to the remap dictionary
    spp_remap.update(
        {name: spp_group for name in spp['SAF_SRM'].unique()}
    )
    del spp

# Apply the remap to create a new grouped species column
grid_evt['fortypnm_gp'] = grid_evt['SAF_SRM'].map(spp_remap).fillna(grid_evt['SAF_SRM'])

# separate the aspen classes
# Override with EVT-based rule for Aspen separation
grid_evt.loc[grid_evt['EVT'] == 7011, 'fortypnm_gp'] = 'Aspen'
grid_evt.loc[grid_evt['EVT'] == 7061, 'fortypnm_gp'] = 'Aspen–Mixed'

# Verify the updated species groups
grid_evt[['grid_idx', 'SAF_SRM', 'fortypnm_gp']].drop_duplicates().head(8)

In [ ]:
# calculate the EVT proportion per gridcell
zs_evt = compute_band_stats(grid_prj, evt_da, id_col='grid_idx', attr='EVT')
# tidy the data frame
zs_evt = zs_evt[['grid_idx', 'EVT', 'pct_cover']]
zs_evt.rename(columns={'pct_cover': 'proportion'}, inplace=True)
zs_evt

In [ ]:
# merge the EVT proportion
grid_evt_pr = grid_evt.merge(zs_evt, on=['grid_idx', 'EVT'])
grid_evt_pr

In [ ]:
# group and recalculate fuel characteristics
grid_evt_rg = (
    grid_evt_pr
    .groupby(['grid_idx', 'fortypnm_gp'])
    .agg({
        'proportion': 'sum',
        'CC': 'mean',
        'CH': 'mean',
        'CBH': 'mean',
        'CBD': 'mean',
        'BALIVE': 'mean',
        'SDI': 'mean',
        'QMD': 'mean'
    })
    .reset_index()
)
grid_evt_rg

In [ ]:
# tidy some of the columns
grid_evt_pr = grid_evt_pr[['grid_idx', 'fortypnm_gp', 'proportion', 'CC', 'CH', 'CBH', 'CBD', 'BALIVE', 'SDI', 'QMD']]
# save this file out.
out_fp = os.path.join(projdir,'data/tabular/mod/gridstats_lf.csv')
grid_evt_pr.to_csv(out_fp)
print(f"Saved file to: {out_fp}")